<a href="https://colab.research.google.com/github/sirmammingtonham/topics_nlp/blob/main/sru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [301]:
class SRUCell(nn.Module):
    def __init__(self, input_size, hidden_size, activation=F.tanh):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.linear = nn.Linear(input_size, hidden_size, bias=False) #W
        self.gate = nn.Linear(input_size, 2*hidden_size, bias=True) #W' and W''
        self.activation = activation

        self.gate_norm = nn.LayerNorm(2 * hidden_size)
        self.c_norm = nn.LayerNorm(hidden_size)
        
        # self.v = nn.Parameter(torch.randn(2*hidden_size))

        self.dropout = nn.Dropout(0.1)

    def forward(self, x, c=None):
        if c is None:
            c = torch.zeros((x.shape[0], self.hidden_size), dtype=x.dtype, device=x.device)

        xt = self.linear(x)
        # gate = torch.sigmoid(self.gate_norm(self.gate(x) + torch.einsum('bs,bs->b', self.v, c)))
        gate = torch.sigmoid(self.gate_norm(self.gate(x)))
        f = gate[:, :, :self.hidden_size]
        r = gate[:, :, self.hidden_size:]
        z = (1.0 - f) * xt + f

        hidden_states = []
        for t in range(x.shape[1]):
            c = f[:, t] * c + z[:, t]
            hidden_states.append(c)

        c_stack = torch.stack(hidden_states, dim=1)
        h = r * self.activation(self.c_norm(c_stack)) + (1.0 - r) * xt
        return h, c

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            val_range = (3.0 / self.input_size) ** 0.5
            module.weight.data.uniform_(-val_range, val_range)
            if module.bias is not None:
                module.bias.data.zero_()

In [380]:
class SRU(nn.Module):
    def __init__(self, num_layers=2, num_digits=16, hidden_dim=128):
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=num_digits, embedding_dim=num_digits)
        self.dropout_embed = nn.Dropout(0.5)
        # self.srus = nn.LSTM(input_size=num_digits, hidden_size=hidden_dim, batch_first=True)#nn.Sequential()
        self.srus = []
        for i in range(num_layers):
            self.srus.append(
                SRUCell(
                    input_size=num_digits if i == 0 else hidden_dim, 
                    hidden_size=hidden_dim
                ).cuda()
            )

        self.linear = nn.Linear(in_features=hidden_dim, out_features=num_digits)
        self.softmax = nn.Softmax(-1)

    def forward(self, x):
        x = self.embed(x)
        x = self.dropout_embed(x)
        c = None
        for sru_layer in self.srus:
            x, c = sru_layer(x, c)

        x, _ = x.max(dim=1)
        x = self.linear(x)
        return self.softmax(x)

In [381]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm

In [382]:
NUM_DIGITS = 16
class CountingDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.int_to_binary(self.data[idx]), self.int_to_binary(self.data[idx] + 1)

    def int_to_binary(self, n):
        binary_list = [int(i) for i in bin(n)[2:]]
        while len(binary_list) < NUM_DIGITS:
            binary_list.insert(0, 0)
        return torch.LongTensor(binary_list)

examples = np.arange(2**NUM_DIGITS-1)
np.random.shuffle(examples)

test_size = 10000
train_ds = CountingDataset(examples[:-test_size])
test_ds = CountingDataset(examples[-test_size:])

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=128, shuffle=True)

In [383]:
model = SRU(num_layers=2, num_digits=16, hidden_dim=128).cuda()
criterion = nn.BCELoss(size_average=True).cuda()
optimizer = torch.optim.Adam(model.parameters())

In [384]:
def evaluate(model, eval_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(tqdm(eval_loader)):
            inputs = inputs.cuda()
            labels = labels.cuda()

            predicted = model(inputs)
            total += labels.shape[0]
            predicted = (predicted > 0.5).float()
            correct += (predicted == labels).all(1).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [385]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs = inputs.cuda()
        labels = labels.cuda()

        outputs = model(inputs)
        loss = criterion(outputs, labels.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    eval_accuracy = evaluate(model, test_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Eval Accuracy: {eval_accuracy:.2f}')

  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch [1/100], Loss: 1.3988, Eval Accuracy: 0.00


  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch [99/100], Loss: 1.3025, Eval Accuracy: 0.00


  0%|          | 0/434 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch [100/100], Loss: 1.3848, Eval Accuracy: 0.00


In [389]:
_, (example, label) = next(enumerate(test_loader))
example = example.cuda()
label = label.cuda()

In [400]:
preds = model(example)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [401]:
preds = (preds > 0.1).int()

In [404]:
example[0]

tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1], device='cuda:0')

In [402]:
preds[0]

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0',
       dtype=torch.int32)